In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display

In [ ]:
# Load the dataset
df = pd.read_csv("spotify_millsongdata.csv")  # Replace with your dataset

# Sample the data to reduce size (optional)
df = df.sample(5000).reset_index(drop=True)

# Basic text cleaning
df['text'] = df['text'].str.lower().replace(r'\\r\\n', ' ', regex=True)

# Drop unnecessary columns
df = df.drop('link', axis=1)

In [ ]:
# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(stop_words='english')

# Fit and transform the text data
tfidf_matrix = tfidf.fit_transform(df['text'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
def recommend(song_name, df, cosine_sim):
    # Find the index of the song
    idx = df[df['song'] == song_name].index[0]
    
    # Get the pairwise similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort the songs based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top 10 similar songs (excluding the song itself)
    sim_scores = sim_scores[1:11]
    
    # Get the song indices
    song_indices = [i[0] for i in sim_scores]
    
    # Return the top 10 similar songs
    return df.iloc[song_indices][['song', 'artist']]

In [ ]:
# Create a dropdown for song selection
song_dropdown = widgets.Dropdown(
    options=df['song'].values,
    description='Select a song:',
    disabled=False,
)

# Create a button to trigger recommendations
recommend_button = widgets.Button(
    description='Show Recommendations',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to get recommendations',
)

# Output widget to display recommendations
output = widgets.Output()

# Function to handle button click
def on_button_click(b):
    with output:
        output.clear_output()  # Clear previous output
        selected_song = song_dropdown.value
        if selected_song:
            recommendations = recommend(selected_song, df, cosine_sim)
            print("### Recommended Songs:")
            for i, row in recommendations.iterrows():
                print(f"**{row['song']}** by **{row['artist']}**")
        else:
            print("Please select a song.")

# Attach the function to the button click event
recommend_button.on_click(on_button_click)

# Display the widgets
display(song_dropdown, recommend_button, output)